In [ ]:
%matplotlib inline
import re
from collections import defaultdict
import numpy as np
from sklearn.decomposition import  LatentDirichletAllocation
from sklearn.feature_extraction.text import CountVectorizer
from nltk.stem.snowball import SnowballStemmer
import matplotlib.pyplot as plt
from sklearn.manifold import TSNE
from sklearn.model_selection import KFold

from prep import *
import pandas as pd
from time import time
import pickle

In [ ]:
"""
load dataset and convert bag of words represenetations.
"""
with open("./text_prep.txt", "r") as t:
    df = t.read()
    df = df.split("\n")

# Set seed
np.random.seed(999)
    
df = pd.DataFrame(df)
df = df[0]
vec = CountVectorizer(min_df = 10, max_features = 10000)
df_vec = vec.fit_transform(df)


In [ ]:
n_topics = 100
alpha = 0.01
beta = 0.1

# 4-fold crossvalidation 
index = list(range(0,len(df)))
kf = KFold(n_splits = 4)

preplexity = []
k = 1
for train_index, test_index in kf.split(index):
    train = df_vec[train_index]
    test = df_vec[test_index]
    lda = LatentDirichletAllocation(n_components = n_topics, doc_topic_prior=alpha, topic_word_prior=beta, n_jobs = 10)
    t0 = time()
    lda.fit(train)

    train_gamma = lda.transform(train)
    train_perplexity = lda.perplexity(train, train_gamma)

    test_gamma = lda.transform(test)
    test_perplexity = lda.perplexity(test, test_gamma)
    
    perplexity.append(test_perplexity)
    k += 1
    print('================================ Fold %f =====================================' %k)


    print('sklearn preplexity: train=%.3f, test=%.3f' %
          (train_perplexity, test_perplexity))

    print("done in %0.3fs." % (time() - t0))
    filename = 'model_fold_{0}.sav'.format(k)
    pickle.dump(lda, open(filename, 'wb'))
    

In [ ]:
avg_perplexity = np.mean(perplexity)